#**Final project, Liliia Yakunina**

##1) Data cleaning



In [ ]:
#  Import libraries
import pandas as pd
import numpy as np
import re
import warnings
from datetime import timedelta, time

from google.colab import files
uploaded = files.upload()

Saving Calls (Done).xlsx to Calls (Done).xlsx
Saving Contacts (Done).xlsx to Contacts (Done) (1).xlsx
Saving Deals (Done).xlsx to Deals (Done) (1).xlsx
Saving Spend (Done).xlsx to Spend (Done) (1).xlsx


#**Dataset  "Contacts"**

In [ ]:
## Contacts: load and preview
contacts = pd.read_excel('Contacts (Done).xlsx', dtype=str)
contacts.info()
contacts.isnull().sum()
contacts.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18548 entries, 0 to 18547
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Id                  18548 non-null  object
 1   Contact Owner Name  18548 non-null  object
 2   Created Time        18548 non-null  object
 3   Modified Time       18548 non-null  object
dtypes: object(4)
memory usage: 579.8+ KB


,Id,Contact Owner Name,Created Time,Modified Time
0,5805028000000645014,Rachel White,27.06.2023 11:28,22.12.2023 13:34
1,5805028000000872003,Charlie Davis,03.07.2023 11:31,21.05.2024 10:23
2,5805028000000889001,Bob Brown,02.07.2023 22:37,21.12.2023 13:17
3,5805028000000907006,Bob Brown,03.07.2023 05:44,29.12.2023 15:20
4,5805028000000939010,Nina Scott,04.07.2023 10:11,16.04.2024 16:14


###**Contacts Table – Notes**

**Client ID (Id)** → rename to Client ID for convenience.

**Manager Name (Contact Owner Name)** → convert to categorical type after cleaning incorrect values (False → unknown).

**Created Time** → convert from object to datetime for time series analysis.

**Modified Time** → convert from object to datetime for tracking contact updates.

Check duplicates and missing values.

Save the cleaned dataset for further analysis.


In [ ]:
#  rename columns
contacts = contacts.rename(columns={'Id': 'Client ID','Contact Owner Name': 'Manager Name'})


In [ ]:
# convert dates
contacts[["Created Time", "Modified Time"]] = contacts[["Created Time", "Modified Time"]].apply(
    pd.to_datetime, errors="coerce")

/tmp/ipython-input-1456495089.py:2: UserWarning: Parsing dates in %d.%m.%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  contacts[["Created Time", "Modified Time"]] = contacts[["Created Time", "Modified Time"]].apply(
/tmp/ipython-input-1456495089.py:2: UserWarning: Parsing dates in %d.%m.%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  contacts[["Created Time", "Modified Time"]] = contacts[["Created Time", "Modified Time"]].apply(


In [ ]:
contacts.head()

,Client ID,Manager Name,Created Time,Modified Time
0,5805028000000645014,Rachel White,2023-06-27 11:28:00,2023-12-22 13:34:00
1,5805028000000872003,Charlie Davis,2023-07-03 11:31:00,2024-05-21 10:23:00
2,5805028000000889001,Bob Brown,2023-07-02 22:37:00,2023-12-21 13:17:00
3,5805028000000907006,Bob Brown,2023-07-03 05:44:00,2023-12-29 15:20:00
4,5805028000000939010,Nina Scott,2023-07-04 10:11:00,2024-04-16 16:14:00


In [ ]:
#  replace invalid values
contacts["Manager Name"] = contacts["Manager Name"].replace("False", "unknown").astype('category')
contacts.info()
contacts["Manager Name"].unique()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18548 entries, 0 to 18547
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Client ID      18548 non-null  object        
 1   Manager Name   18548 non-null  category      
 2   Created Time   18548 non-null  datetime64[ns]
 3   Modified Time  18548 non-null  datetime64[ns]
dtypes: category(1), datetime64[ns](2), object(1)
memory usage: 454.2+ KB


['Rachel White', 'Charlie Davis', 'Bob Brown', 'Nina Scott', 'Alice Johnson', ..., 'Zachary Foster', 'Sam Young', 'Wendy Clark', 'Tina Zhang', 'Derek James']
Length: 28
Categories (28, object): ['Alice Johnson', 'Amy Green', 'Ben Hall', 'Bob Brown', ..., 'Wendy Clark',
                          'Yara Edwards', 'Zachary Foster', 'unknown']

In [ ]:
# save
# contacts.to_excel("Сontacts_cleaned.xlsx", index=False)
# files.download("/content/Сontacts_cleaned.xlsx")

In [ ]:
# contacts.to_parquet("Contacts_cleaned.parquet", index=False)
# files.download("/content/Contacts_cleaned.parquet")

#**Dataset "Calls"**

In [ ]:
# Calls:  load and clean

calls  = pd.read_excel('Calls (Done).xlsx', dtype=str)
calls.info()
calls.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95874 entries, 0 to 95873
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Id                          95874 non-null  object
 1   Call Start Time             95874 non-null  object
 2   Call Owner Name             95874 non-null  object
 3   CONTACTID                   91941 non-null  object
 4   Call Type                   95874 non-null  object
 5   Call Duration (in seconds)  95791 non-null  object
 6   Call Status                 95874 non-null  object
 7   Dialled Number              0 non-null      object
 8   Outgoing Call Status        86875 non-null  object
 9   Scheduled in CRM            86875 non-null  object
 10  Tag                         0 non-null      object
dtypes: object(11)
memory usage: 8.0+ MB


,0
Id,0
Call Start Time,0
Call Owner Name,0
CONTACTID,3933
Call Type,0
Call Duration (in seconds),83
Call Status,0
Dialled Number,95874
Outgoing Call Status,8999
Scheduled in CRM,8999


###**Data types analysis and recommendations**:

**Call Start Time**: object → convert to datetime for time series analysis.

**Manager Name (Call Owner Name)**: object → convert to categorical type.

**Client ID (CONTACTID)**: object → convert to string (or int if numeric) since it’s an identifier.

**Call Type**: object → convert to categorical type.

**Call Duration sec (Call Duration (in seconds))**: object → convert to numeric (int32) and fill missing values with median.

**Call Status**: object → convert to categorical type.

**Outgoing Call Status**: object → convert to categorical type and fill missing values with "unknown".

**Scheduled in CRM**: object → convert to categorical type with values "Scheduled" / "not_Scheduled" and fill missing values with "unknown".

###**Missing values analysis and handling**:

**Call Duration sec** → missing values filled with median to better represent typical call durations.

**Dialled Number** → dropped because it’s not used (all values empty).

**Tag** → dropped because it’s not used (all values empty).

**Outgoing Call Status** → missing values filled with "unknown" to keep structure and not lose rows.

**Scheduled in CRM** → missing values filled with "unknown" to explicitly indicate unknown scheduling.

**Client ID** → rows with missing values dropped because this identifier is required for joining with other tables.

In [ ]:
#  duplicates
print(f'Full duplicates: {calls.duplicated().sum()}')
print(f'Missing values:\n {calls.isnull().sum()}')


Full duplicates: 0
Missing values:
 Id                                0
Call Start Time                   0
Call Owner Name                   0
CONTACTID                      3933
Call Type                         0
Call Duration (in seconds)       83
Call Status                       0
Dialled Number                95874
Outgoing Call Status           8999
Scheduled in CRM               8999
Tag                           95874
dtype: int64


In [ ]:
#  rename columns
calls.rename(columns={
     'Id': 'Call ID',
     'Call Owner Name': 'Manager Name',
     'CONTACTID': 'Client ID',
     'Call Duration (in seconds)': 'Call Duration sec'
 }, inplace=True)


In [ ]:
#  drop unnecessary columns
calls = calls.drop(columns=["Dialled Number", "Tag"])


In [ ]:
#  convert call duration
calls['Call Duration sec'] = pd.to_numeric(calls['Call Duration sec'], errors='coerce')
calls['Call Duration sec'] = calls['Call Duration sec'].fillna(calls['Call Duration sec'].median()).astype('int32')



In [ ]:
#  convert datetime
calls['Call Start Time'] = pd.to_datetime(calls['Call Start Time'], errors='coerce')


/tmp/ipython-input-1781580626.py:2: UserWarning: Parsing dates in %d.%m.%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  calls['Call Start Time'] = pd.to_datetime(calls['Call Start Time'], errors='coerce')


In [ ]:
#  Creating an aggregated status
mapping = {
    'Attended Dialled': 'Successful',
    'Received': 'Successful',
    'Unattended Dialled': 'Unsuccessful',
    'Missed': 'Unsuccessful',
    'Overdue': 'Unsuccessful',
    'Scheduled Attended Delay': 'Other',
    'Scheduled Unattended Delay': 'Other',
    'Cancelled': 'Other',
    'Scheduled Attended': 'Other',
    'Scheduled Unattended': 'Other',
    'Scheduled': 'Other'}

calls['Call Status Grouped'] = calls['Call Status'].map(mapping)


In [ ]:
# Scheduled / replace boolean
calls["Scheduled in CRM"] = calls["Scheduled in CRM"].replace({"True": "Scheduled", "False": "not_Scheduled"})


In [ ]:
#  clean missing
cols_to_fix = ['Call ID', 'Client ID', 'Manager Name', 'Call Type','Call Status', 'Outgoing Call Status', "Scheduled in CRM"]
for col in cols_to_fix:
    calls[col] = calls[col].astype(str).replace(r'^\s*$', 'unknown', regex=True).replace('nan', 'unknown')


In [ ]:
# categorical columns
category_columns = ['Manager Name', 'Call Type', 'Call Status', 'Outgoing Call Status', 'Scheduled in CRM', 'Call Status Grouped']
for col in category_columns:
    if col in calls.columns:
        calls[col] = calls[col].astype("category")
print(calls.dtypes)

Call ID                         object
Call Start Time         datetime64[ns]
Manager Name                  category
Client ID                       object
Call Type                     category
Call Duration sec                int32
Call Status                   category
Outgoing Call Status          category
Scheduled in CRM              category
Call Status Grouped           category
dtype: object


In [ ]:
# save exel

# calls.to_excel("Сalls_cleaned.xlsx", index=False)
# files.download("/content/Сalls_cleaned.xlsx")

In [ ]:
# calls.to_parquet("Calls_cleaned.parquet", index=False)
# files.download("/content/Calls_cleaned.parquet")

#**Dataset "Spend"**

In [ ]:
# Spend: load and clean
spend = pd.read_excel("Spend (Done).xlsx", dtype=str)
print(spend.head())
spend.info()

                  Date        Source               Campaign Impressions Spend  \
0  2023-07-03 00:00:00    Google Ads         gen_analyst_DE           6     0   
1  2023-07-03 00:00:00    Google Ads  performancemax_eng_DE           4  0.01   
2  2023-07-03 00:00:00  Facebook Ads                    NaN           0     0   
3  2023-07-03 00:00:00    Google Ads                    NaN           0     0   
4  2023-07-03 00:00:00           CRM                    NaN           0     0   

  Clicks AdGroup   Ad  
0      0     NaN  NaN  
1      1     NaN  NaN  
2      0     NaN  NaN  
3      0     NaN  NaN  
4      0     NaN  NaN  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20779 entries, 0 to 20778
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Date         20779 non-null  object
 1   Source       20779 non-null  object
 2   Campaign     14785 non-null  object
 3   Impressions  20779 non-null  object
 4   Spend     

###**Data Type Analysis and Recommendations:**

**Ad Channel (Source)** → convert to categorical type for efficient storage and analysis.

**Campaign Name (Campaign)** → convert to categorical type.

**Impressions Count (Impressions)** → convert to numeric type Int32.

**Clicks Count (Clicks)** → convert to numeric type Int32.

**Spend Amount(€) (Spend)** → convert to numeric type float for calculations and analysis.

**Ad Group and Ad Name** → convert to string and optionally categorical type.


###**Missing Values Analysis and Handling:**

**Campaign Name** → fill missing values with "unknown" to retain information about the campaign even if not provided.

**Ad Group** → optional: can keep or drop if it contains many missing values and is not critical for analysis.

**Ad Name** → fill missing values with "unknown" or drop if necessary.

**Impressions Count, Clicks Count, Spend Amount(€)** → replace missing values with 0 to ensure correct aggregation and ROI calculations.

In [ ]:
# Count of unique values in a column
num_unique1 = spend['AdGroup'].nunique()
print(num_unique1)

num_unique1 = spend['Ad'].nunique()
print(num_unique1)


24
176


In [ ]:
spend['AdGroup'].unique()

array([nan, 'women', 'wide', 'interest_programming', 'recentlymoved',
       'interest_dataanalytics', 'interest_work',
       'interest_programming – Copy', 'interest_dataanalytics – Copy',
       'LAL1', 'b', 'Com_july_1', 'interest_all', 'Com_august',
       'interest_work_WebDev', 'interest_programming_WebDev',
       'promoposts_b', 'retargeting', 'wide_webdesigner',
       'wide_python-developer', 'wide_qa-engineer',
       'interest_python-developer', 'berlin_wide', 'Com_march',
       'accountant_wide'], dtype=object)

In [ ]:
spend['Ad'].unique()

array([nan, 'b3', 'b1', 'b4', 'b2', 'v2', 'v1', 'b4com', 'b3com', 'b2com',
       'b1com', 'v6com', 'v5', 'v4com', 'v3com', 'v5com', 'ad4', 'ad1',
       'ad2', 'ad3', 'v8com', 'v7com', 'ad6', 'ad5', 'bloggersvideo1com',
       'v9com', 'ad9', 'ad8', 'ad_blogger_1', 'ad_blogger_2', 'ad7',
       'web_b3', 'web_b5', 'web_b1', 'web_b4', 'web_b2', 'ad_blogger_3',
       'v10com', 'bloggersvideo2com', 'b5', 'b6', 'b8', 'b7', 'v3', 'v10',
       'v12', 'v11com', 'v11', 'ad_gov_1', 'ad_da_1', 'b3comwebdev',
       'bloggersvideo2comwebdev', 'v11comwebdev', 'b1comwebdev',
       'b2comwebdev', 'bloggersvideo4com', 'bloggersvideo3com',
       'bloggersvideo5', 'promo2', 'promo1', 'ad_blogger_4',
       'bloggersvideo4', 'b10', 'b11', 'b12', 'ad_blogger_6', 'promo3',
       'b15blackfriday', 'b14blackfriday', 'b13blackfriday', 'b7webinar',
       'b6webinar', 'b4webinar', 'b5webinar', 'bloggersvideo6blackfriday',
       'bloggersvideo6webinar', 'bloggersvideo7blackfriday',
       'bloggersvideo

In [ ]:
# Function to normalize and clean the spend DataFrame
def normalize_spend(spend_df):

     # Sub-function: normalize any text (lowercase, remove extra spaces, replace empty with 'unknown')
    def normalize_text(text):
        if pd.isna(text) or str(text).strip() == '':
            return 'unknown'
        text = str(text).lower()
        text = text.replace('_', ' ')
        text = re.sub(r'\s+', ' ', text)
        return text.strip()

    # Sub-function: special cleaning rules for 'AdGroup' column
    def clean_adgroup(text):
        if pd.isna(text) or str(text).strip() == '':
            return 'unknown'
        text = text.lower().replace('dataanalytics','data analytics')\
                           .replace('python-developer','python developer')\
                           .replace('programming – copy','programming')\
                           .replace('data analytics – copy','data analytics')
        return re.sub(r'\s+', ' ', text).strip()

    # Apply normalization to AdGroup (both generic + specific cleaning)
    spend_df['AdGroup'] = spend_df['AdGroup'].apply(normalize_text).apply(clean_adgroup)

    # Apply normalization only to Ad column
    spend_df['Ad'] = spend_df['Ad'].apply(normalize_text)

    # Replace all NaN with 'unknown'
    spend_df.fillna('unknown', inplace=True)

    return spend_df

# Run normalization on the spend dataset
spend_normalized = normalize_spend(spend)


In [ ]:
# Get unique values
spend['AdGroup'].unique()

array(['unknown', 'women', 'wide', 'interest programming',
       'recentlymoved', 'interest data analytics', 'interest work',
       'lal1', 'b', 'com july 1', 'interest all', 'com august',
       'interest work webdev', 'interest programming webdev',
       'promoposts b', 'retargeting', 'wide webdesigner',
       'wide python developer', 'wide qa-engineer',
       'interest python developer', 'berlin wide', 'com march',
       'accountant wide'], dtype=object)

In [ ]:
# Get unique values
spend['Ad'].unique()

array(['unknown', 'b3', 'b1', 'b4', 'b2', 'v2', 'v1', 'b4com', 'b3com',
       'b2com', 'b1com', 'v6com', 'v5', 'v4com', 'v3com', 'v5com', 'ad4',
       'ad1', 'ad2', 'ad3', 'v8com', 'v7com', 'ad6', 'ad5',
       'bloggersvideo1com', 'v9com', 'ad9', 'ad8', 'ad blogger 1',
       'ad blogger 2', 'ad7', 'web b3', 'web b5', 'web b1', 'web b4',
       'web b2', 'ad blogger 3', 'v10com', 'bloggersvideo2com', 'b5',
       'b6', 'b8', 'b7', 'v3', 'v10', 'v12', 'v11com', 'v11', 'ad gov 1',
       'ad da 1', 'b3comwebdev', 'bloggersvideo2comwebdev',
       'v11comwebdev', 'b1comwebdev', 'b2comwebdev', 'bloggersvideo4com',
       'bloggersvideo3com', 'bloggersvideo5', 'promo2', 'promo1',
       'ad blogger 4', 'bloggersvideo4', 'b10', 'b11', 'b12',
       'ad blogger 6', 'promo3', 'b15blackfriday', 'b14blackfriday',
       'b13blackfriday', 'b7webinar', 'b6webinar', 'b4webinar',
       'b5webinar', 'bloggersvideo6blackfriday', 'bloggersvideo6webinar',
       'bloggersvideo7blackfriday', 'blogger

In [ ]:
#  missing
print("Missing:\n", spend.isnull().sum())

# duplicates
dublicate_spend = spend.duplicated().sum()
print(f"Number of complete duplicate rows: {dublicate_spend}")

Missing:
 Date           0
Source         0
Campaign       0
Impressions    0
Spend          0
Clicks         0
AdGroup        0
Ad             0
dtype: int64
Number of complete duplicate rows: 917


In [ ]:
# drop duplicates
spend = spend.drop_duplicates()
print(f"How many full duplicate rows remain after cleaning: {spend.duplicated().sum()}")

How many full duplicate rows remain after cleaning: 0


In [ ]:
# rename
spend.rename(columns={
    'Date': 'Tracking Date',
    'Source': 'Lead Source',
    'Campaign': 'Campaign Name',
    'Impressions': 'Impressions Count',
    'Spend': 'Spend Amount(€)',
    'Clicks': 'Clicks Count',
    'AdGroup': 'Ad Group',
    'Ad': 'Ad Name'
}, inplace=True)


/tmp/ipython-input-3034229637.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spend.rename(columns={


In [ ]:
# column types
spend = spend.astype({"Lead Source": "category","Campaign Name": "string","Ad Group": "string","Ad Name": "string"})


In [ ]:
#  fill missing
for col in ["Campaign Name"]:
    spend[col].fillna("unknown", inplace=True)


/tmp/ipython-input-1931858060.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  spend[col].fillna("unknown", inplace=True)


In [ ]:
# numeric columns
spend["Impressions Count"] = pd.to_numeric(spend["Impressions Count"], errors="coerce").fillna(0).astype("Int64")
spend["Clicks Count"] = pd.to_numeric(spend["Clicks Count"], errors="coerce").fillna(0).astype("Int64")
spend["Spend Amount(€)"] = pd.to_numeric(spend["Spend Amount(€)"], errors="coerce").fillna(0).astype(float)
spend["Tracking Date"] = pd.to_datetime(spend["Tracking Date"], errors="coerce").dt.normalize()
spend.info()


<class 'pandas.core.frame.DataFrame'>
Index: 19862 entries, 0 to 20778
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Tracking Date      19862 non-null  datetime64[ns]
 1   Lead Source        19862 non-null  category      
 2   Campaign Name      19862 non-null  string        
 3   Impressions Count  19862 non-null  Int64         
 4   Spend Amount(€)    19862 non-null  float64       
 5   Clicks Count       19862 non-null  Int64         
 6   Ad Group           19862 non-null  string        
 7   Ad Name            19862 non-null  string        
dtypes: Int64(2), category(1), datetime64[ns](1), float64(1), string(3)
memory usage: 1.3 MB


In [ ]:
spend.head()

,Tracking Date,Lead Source,Campaign Name,Impressions Count,Spend Amount(€),Clicks Count,Ad Group,Ad Name
0,2023-07-03,Google Ads,gen_analyst_DE,6,0.00,0,unknown,unknown
1,2023-07-03,Google Ads,performancemax_eng_DE,4,0.01,1,unknown,unknown
2,2023-07-03,Facebook Ads,unknown,0,0.00,0,unknown,unknown
3,2023-07-03,Google Ads,unknown,0,0.00,0,unknown,unknown
4,2023-07-03,CRM,unknown,0,0.00,0,unknown,unknown


In [ ]:
# #  save
# spend.to_excel("Spend_cleaned.xlsx", index=False)

# files.download("/content/Spend_cleaned.xlsx")

In [ ]:
# spend.to_parquet("Spend_cleaned.parquet", index=False)
# files.download("/content/Spend_cleaned.parquet")

##**Dataset "Deals"**

In [ ]:
# Read Excel as strings
deals = pd.read_excel('Deals (Done).xlsx', dtype=str)

#  Show last 5 rows
print(deals.tail())

# Count missing values per column
print("Missing:\n", deals.isnull().sum())

# Count full duplicates
print(f"Full duplicates: {deals.duplicated().sum()}")

                        Id Deal Owner Name Closing Date            Quality  \
21590  5805028000000945016      Jane Smith   29.08.2023           A - High   
21591  5805028000000927004       Bob Brown   09.07.2023     D - Non Target   
21592  5805028000000922001       Bob Brown   03.07.2023  E - Non Qualified   
21593                  NaN             NaN          NaN                NaN   
21594                  NaN             NaN          NaN                NaN   

      Stage             Lost Reason                   Page         Campaign  \
21590  Lost        Changed Decision  eng/digital-marketing  02.07.23wide_DE   
21591  Lost  Does not speak English  eng/digital-marketing    03.07.23women   
21592  Lost                 Refugee                      /              NaN   
21593   NaN                     NaN                    NaN              NaN   
21594   NaN                     NaN                    NaN              NaN   

                     SLA Content  ... Product Education 

In [ ]:
# Normalize and clean deal data: standardize terms, extract dates, and clean text fields
def normalize_deals(deals_df):
    import re
    import pandas as pd

    # Mapping of various term variants to standardized terms
    TERM_MAP = {
        **dict.fromkeys(['it career hub', 'it hub', 'itcareerhub', 'it%20career%20hub', 'ich','айти карьер хаб'], 'it career hub'),
        **dict.fromkeys(['data analyst','курсы data analyst', 'data analyst курсы', 'data analyst обучение',
                         'analytics', 'data analysis analytics', 'аналитика данных с нуля',
                         'аналитик данных обучение', 'аналитиквitобучение','дата аналитик курсы',
                         'дата аналитик это', 'дата аналитика', 'курсы аналитик данных',
                         'курсы аналитика', 'курсы аналитика с нуля','курсы по дата аналитике',
                         'обучение на аналитика данных'], 'data analyst'),
        **dict.fromkeys(['lost does not answer', 'ntc lost','allcomlost','allthinking lost','all'], 'lost'),
        **dict.fromkeys(['wide python-developer','python developer'], 'python developer'),
        **dict.fromkeys(['invitation', 'invitation\\'], 'invitation'),
    }

    # Extract date from text in format dd_mm_yyyy
    def extract_date(text):
        match = re.search(r'\b\d{1,2}_\d{1,2}_\d{4}\b', str(text)) if pd.notna(text) else None
        if match:
            return pd.to_datetime(match.group(0), format='%d_%m_%Y', errors='coerce')
        return pd.NaT

    # Clean and normalize text
    def normalize_text(text):
        if pd.isna(text) or str(text).strip() == '':  # handle empty values
            return 'unknown'
        text = str(text)
        text = re.sub(r'http\S+', '', text)  # remove URLs
        text = re.sub(r'\b\d{1,2}_\d{1,2}_\d{4}\b', '', text)  # remove dates
        text = re.sub(r'recentlymoved-?', 'recently moved', text)  # fix typo
        text = re.sub(r'it[_ ]career[_ ]hub[_ ]?', 'it career hub', text)  # standardize term
        text = re.sub(r'it[_ ]hub', 'it hub', text)
        text = text.replace('_', ' ').lower()  # lowercase and replace underscores
        text = re.sub(r'\s+', ' ', text)  # remove extra spaces
        return text.strip() if text.strip() else 'unknown'

    # Map text to standardized terms
    def semantic_normalize(text):
        return TERM_MAP.get(text, text)

    # Apply functions to DataFrame columns
    deals_df['Term Date'] = deals_df['Term'].apply(extract_date)
    deals_df['Term_norm'] = deals_df['Term'].apply(normalize_text).apply(semantic_normalize)
    deals_df['Content_norm'] = deals_df['Content'].apply(normalize_text)

    return deals_df

# Normalize deals DataFrame
deals_normalized = normalize_deals(deals)


###**Data Type Analysis and Recommendations:**

**Id:** float64 → Int64, because it is an identifier.

**Deal Owner Name**: object → categorical type.

**Closing Date: object** → datetime for time series analysis.

**Quality**: object → categorical type.

**Stage**: object → categorical type.

**Lost Reason**: object → categorical type.

**Page:** object → categorical type.

**Campaign**: object → categorical type.

**SLA**: object → convert to seconds (using a function).

**Content**: object → categorical type.

**Term**: object → categorical type.

**Source**: object → categorical type.

**Payment Type**: object → categorical type.

**Product**: object → categorical type.

**Education Type**: object → categorical type.

**Created Time**: object → datetime for time series analysis.

**Course duration**: float64 → Int8.

**Months of study**: float64 → Int8.

**Initial Amount Paid**: object → Int32 for financial analysis (handle non-numeric values).

**Offer Total Amount**: object → Int32 for financial analysis (handle non-numeric values).

**Contact Name**: float64 → Int64, because it is an identifier.

**City**: object → categorical type.

**Level of Deutsch**: object → categorical type.

###**Missing Values Analysis and Recommendations:**

**Id **– drop missing values, as it is an identifier and must be filled for correct joins with other tables.

**Deal Owner Name **– fill missing values with "Unknown".

**Closing Date** – keep missing values as they may indicate the deal is still open or the closing date was not recorded. Convert the column to datetime using errors='coerce', so missing values remain as NaT.

**Quality** – fill missing values with "Unknown".

**Stage** – fill missing values with "Unknown".

**Lost Reason** – fill missing values with "Unknown".

**Page** – fill missing values with "Unknown".

**Campaign** – fill missing values with "Unknown".

**SLA** – fill missing values with "Unknown".

**Content** – fill missing values with "Unknown".

**Term** – fill missing values with "Unknown".

**Source** – fill missing values with "Unknown".

**Payment Type** – fill missing values with "Unknown".

**Product** – fill missing values with "Unknown".

**Course duration** – fill missing values with the median value.

**Months of study** – fill missing values with the median value.

**Initial Amount Paid** – fill missing values with the median value.

Offer Total Amount – fill missing values with the median value.

**Contact Name** – drop missing values, as it is an identifier and must be filled for correct joins.

City – fill missing values with "Unknown".

**Level of Deutsch** – fill missing values with "Unknown".

**Education Type** – fill missing values with "Unknown".

In [ ]:
# Переименование / rename
deals.rename(columns={
    'Id': 'Deal ID',
    'Deal Owner Name': "Manager Name",
    'Stage': 'Deal Stage',
    'Page': 'Landing Page',
    'Campaign': 'Campaign Name',
    'Content': 'Ad Name',
    'Term': 'Ad Group',
    'Source': 'Lead Source',
    'Product': 'Course Name',
    'Contact Name': 'Client ID',
    'Level of Deutsch': 'German Level'
}, inplace=True)


In [ ]:
# Convert time string "HH:MM:SS" to total seconds
def time_str_to_seconds(time_str) -> int:
    if pd.isna(time_str) or str(time_str).strip() == "":  # handle empty or NaN values
        return 0
    try:
        clean_str = str(time_str).strip().rstrip("-")  # remove trailing "-" and spaces
        hours, minutes, seconds = map(int, clean_str.split(":"))  # split into H, M, S
        return hours * 3600 + minutes * 60 + seconds  # calculate total seconds
    except:
        return 0  # return 0 if parsing fails

# Apply conversion to the 'SLA' column
deals['SLA'] = deals['SLA'].apply(time_str_to_seconds)


In [ ]:
# Dates processing
# Convert columns to datetime format
deals['Created Time'] = pd.to_datetime(deals['Created Time'], format="%d.%m.%Y %H:%M", errors='coerce')
deals['Closing Date'] = pd.to_datetime(deals['Closing Date'], format="%d.%m.%Y", errors='coerce')
deals['Term Date'] = pd.to_datetime(deals['Term Date'], format="%d.%m.%Y", errors='coerce')

# Fill missing 'Closing Date' with 'Term Date'
deals['Closing Date'] = deals['Closing Date'].fillna(deals['Term Date'])

# Drop 'Term Date' column
deals.drop(columns=['Term Date'], inplace=True)


In [ ]:
# Function to clean and convert money values into EUR

def clean_money_eur(df, col):
    df[col] = (df[col].astype(str)
               .str.replace(r'[€$]', '', regex=True)
               .str.replace('.', '', regex=False)
               .str.replace(',', '.', regex=False)
               .str.strip())
    df[col] = pd.to_numeric(df[col], errors='coerce')
    if df[col].notna().any():
        df.rename(columns={col: f"{col} (EUR)"}, inplace=True)

for c in ["Initial Amount Paid", "Offer Total Amount"]:
    clean_money_eur(deals, c)


In [ ]:
#  Normalize German level
A0_list = [
    'нулевой уровень, только пошел на курсы.', 'Нет', 0,
    'ня-0, но англ B2+', 'не учил', 'никакой', 'идет на А1'
]

A2_list = [
    'B1 (почти, не сдала чуть) + англ В1'
]

B1_list = [
    'b2 (ждет серт)', '5 июля 2024 сдает экз на В2', 'B2 (ждет итог экзамена)',
    'В2 - не сдал', 'B2+ (не сдавал, но говорит)', 'b2 ждет серт', 'В январе - В2 сдает'
]

unknow_list = [
    'B1 будет в феврале 2025', 'Detmold, Paulinenstraße 95, 32756',
    'Сам оценивает на B2, 13 лет живет в Германии',
    'lэкзамен - 6 июля на В1. курсы вечером (но уверенно говорит на B1)',
    'Гражданка Германии уже год в Германии Учит немецкий и в сентябре b1 через гос-во проходит, а не через ДЖЦ, вечером учится 3 р в неделю с 18 до 21',
    '-', 'B1 в процессе обучения',
    'ЯЗ: нем В1 был экз 03.05 повтор и сейчас ждет результаты. Технический англ был. А1 сейчас. ОБР: 2 во информационные и комп сети - инженер системоте',
    'В1 в сентябре', 'Ждем B1', 'окончание 13.06 курса на b1',
    'Thorn-Prikker-Str. 30, Hagen, 58093', 'Ждет результат по B1',
    'b1 экзамен будет 12 апреля','ждем B1','Paderborn 33102, Schwabenweg 10',
    'Ждем B1 со дня на день', 'B1 еще нет результата','31.05.2024',
    'Lichtenfelser Straße 25, Untersiemau 96253','Учиться до сентября на B1',
    'b1 9ждет экзамен)','гражданка','b1 (ждет результат)','Гражданин','25 лет живет в Германии',
    'Ждем результат по B1','Ждет со дня на день В1','b1 (ждет результаты)',
    'ждет сертификат B1', 'b1 ждет серт на днях на руки','b1 24 февраля экзамен, англ b2',
    'А2 ( скоро екзамен)','не сдавал, но гражданин','Нет сертификатов, но есть С1 англ, неоконченное высшее в ИТ (и еще одно высшее юридическое) , очень хочет в ИТ, сильно замотивирована именно н',
    'учит A2','Б1 ( ждет итог )','б1 (ждет рез-тат)','В','будет B1 в июне',
    'курс А2-В1 - сдача в июле, но вечерняя смена инт курсов, настроен получить гутшайн уже сейчас.',
    'Б1( может будет)','b1 ждет результаты','b1 ждет экзамен в феврале','Без 5 минут B1 (ждет результаты экзамена)',
    'Проходит сейчас B1','Ждет результат по B1 в феврале','Проходит сейчас повторно B1','b1 экзамен в феврале',
    'Учиться на B1 во вторую смену, в первую хочет получить одобрение на обучение у нас','Б1?','?','b1 экзамен 26 января',
    'f2','Учиться на B1','Сдала экзамен на B1, ждет в начале февраля результат','В январе будут результаты по экзамену на B1',
    'ня-0, ая-B1','в1 ждем результаты','B1 (ждет результаты)','не учила ( разговорный) сразу пошла работать',
    'Б1 ( был екзамен ждет итог )','ня а2, ая в1','в1, еще нет сертификата','Бй','ждет результаты по B1 экзамену',
    90,'.','b1 результат экзамена в феврале','УТОЧНИТЬ!','B1 (до февраля)','C','б1 заканчивает','В1?',
    'b1 будет в январе экзамен, готов совмещать','A','b1 (ждет серт)','УТОЧНИТЬ','разговорный из украины, без сертификата',
    'Ждет B1','сдавала А2 в сентябре','точно уровень не знаю, но говорить могу - учила сама','b1 должна получить результаты в феврале'
]

# level_dict: dictionary for manual normalization of levels A0, A2, B1, and unknown
def normalize_level(text):
    if pd.isna(text) or str(text).strip() == "":
        return "unknown"
    text_str = str(text).lower()

 #  check against predefined lists
    if any(str(x).lower() == text_str for x in A0_list):
        return "A0"
    if any(str(x).lower() == text_str for x in A2_list):
        return "A2"
    if any(str(x).lower() == text_str for x in B1_list):
        return "B1"
    if any(str(x).lower() == text_str for x in unknow_list):
        return "unknown"
    if "нулевой" in text_str:
        return "A0"

    #  replace Cyrillic letters with Latin
    mapping = {"а":"a", "б":"b", "в":"b", "с":"c", "ё":"e"}
    for k, v in mapping.items():
        text_str = text_str.replace(k, v)

    #  exact level match A0-C2
    match = re.search(r"\b(a0|a1|a2|b1|b2|c1|c2)\b", text_str)
    if match:
        return match.group(1).upper()
    return "unknown"

# apply function to column
deals['German Level New'] = deals['German Level'].apply(normalize_level)


In [ ]:
# drop unnecessary rows/cols
deals = deals.drop([21593, 21594])
deals = deals.drop(columns=['German Level', 'Months of study'])


In [ ]:
# Fill missing categorical values
fillna_columns = [
    'Manager Name', 'Lost Reason', 'Landing Page','Campaign Name', 'Quality', 'Ad Name', 'Ad Group',
    'Payment Type', 'Course Name', 'Education Type', 'Client ID', 'City'
]

for col in fillna_columns:
    if col in deals.columns:
        deals[col] = deals[col].astype("string")  # ensure column is string type

        # Replace empty strings, "-", "/", or spaces with 'unknown'
        deals[col] = deals[col].replace(r'^\s*[-/]*\s*$', 'unknown', regex=True)

        # Fill NaN values with 'unknown'
        deals[col] = deals[col].fillna('unknown')


In [ ]:
#  column types
deals = deals.astype({'Deal ID':'string','Manager Name':'category','Deal Stage':'category',
                      'Lost Reason':'category','Quality':'category', 'Lead Source':'category',
                      'Payment Type':'category',
                      'Course Name':'category','Education Type':'category','Landing Page':'string',
                      'Campaign Name':'string','Ad Name':'string','Ad Group':'string',
                      'Course duration':'Int64','Initial Amount Paid (EUR)':'Int64',
                      'Offer Total Amount (EUR)':'Int64','City':'string','German Level New':'category'})


In [ ]:
#  fix initial amount
def replace_if_greater(df, col1, col2):
    """
 Compares values in two columns row by row.
 If the value in col1 is greater than col2, it replaces col1 with col2.
 Returns a pd.DataFrame with the updated col1 column.\

    \Сравнивает значения в двух колонках построчно.
    Если значение в col1 > col2, заменяет col1 = col2.
    Возвращает  pd.DataFrame: DataFrame с обновленной колонкой col1
    """
    df[col1] = df[col1].where(df[col1] <= df[col2], df[col2])
    return df
deals = replace_if_greater(deals, "Initial Amount Paid (EUR)", "Offer Total Amount (EUR)")


In [ ]:
# Fill missing values with the median for numeric columns

median_columns = ['Course duration', 'Initial Amount Paid (EUR)', 'Offer Total Amount (EUR)']
for col in median_columns:
    deals[col] = deals[col].fillna(deals[col].median())

In [ ]:
#  remove lost duplicates
deals = deals.drop(deals[(deals['Deal Stage']=='Lost') & (deals['Lost Reason']=='Duplicate')].index)


In [ ]:
#  Payment Status
deals["Payment Status"] = np.where(deals["Closing Date"].notna() & (deals["Deal Stage"]=="Payment Done"), "Paid", "Unpaid")


In [ ]:
#  rename course
deals["Course Name"] = deals["Course Name"].replace("Web Developer", "Web Development")


/tmp/ipython-input-208339850.py:2: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  deals["Course Name"] = deals["Course Name"].replace("Web Developer", "Web Development")


In [ ]:
# Clean and normalize City column

def extract_city(value):
    """Extract city from string"""
    if pd.isna(value) or str(value).strip() in ["-", ""]:
        return "unknown"
    parts = [p.strip() for p in str(value).split(",") if p.strip()]  # split by comma and strip
    if len(parts) == 0:
        return "unknown"
    # Choose second part if it looks like a city, otherwise last part
    if len(parts) >= 2 and parts[1].isalpha():
        return parts[1]
    return parts[-1]

# Apply city extraction function
deals["City"] = deals["City"].apply(extract_city)

# Fix known hyphen issues
deals["City"] = deals["City"].replace({"Villingen\u2011Schwenningen": "Villingen-Schwenningen"})


In [ ]:
deals.head()

,Deal ID,Manager Name,Closing Date,Quality,Deal Stage,Lost Reason,Landing Page,Campaign Name,SLA,Ad Name,...,Created Time,Course duration,Initial Amount Paid (EUR),Offer Total Amount (EUR),Client ID,City,Term_norm,Content_norm,German Level New,Payment Status
0,5805028000056864695,Ben Hall,NaT,unknown,New Lead,unknown,/eng/test,03.07.23women,0,v16,...,2024-06-21 15:30:00,11,1000,11000,5805028000056849495,unknown,women,v16,unknown,Unpaid
1,5805028000056859489,Ulysses Adams,NaT,unknown,New Lead,unknown,/at-eng,unknown,0,unknown,...,2024-06-21 15:23:00,6,0,2000,5805028000056834471,unknown,unknown,unknown,unknown,Unpaid
2,5805028000056832357,Ulysses Adams,2024-06-21,D - Non Target,Lost,Non target,/at-eng,engwien_AT,1603,b1-at,...,2024-06-21 14:45:00,11,1000,11000,5805028000056854421,unknown,unknown,b1-at,unknown,Unpaid
3,5805028000056824246,Eva Kent,2024-06-21,E - Non Qualified,Lost,Invalid number,/eng,04.07.23recentlymoved_DE,3604,bloggersvideo14com,...,2024-06-21 13:32:00,11,1000,11000,5805028000056889351,unknown,recently moved,bloggersvideo14com,unknown,Unpaid
4,5805028000056873292,Ben Hall,2024-06-21,D - Non Target,Lost,Non target,/eng,discovery_DE,3192,website,...,2024-06-21 13:21:00,11,1000,11000,5805028000056876176,unknown,unknown,website,unknown,Unpaid


In [ ]:
# save cleaned file Deals
# deals.to_excel("Deals_cleaned.xlsx", index=False)
# # print("Saved: Deals_cleaned.xlsx")
# files.download("/content/Deals_cleaned.xlsx")


In [ ]:
# deals.to_parquet("Deals_cleaned.parquet", index=False)
# files.download("/content/Deals_cleaned.parquet")